In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sales_train = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales_eval  = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
calendar    = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
prices      = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')

print('train_shape   :' ,sales_train.shape)
print('eval_shape    :' ,sales_eval.shape)
print('calendar_shape:' ,calendar.shape)
print('prices_shape  :' ,prices.shape)

In [ ]:
sales_train['state_id'].unique()

In [ ]:
sales_train.head()

In [ ]:
calendar.tail()

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

fig = plt.figure()

msno.bar(calendar)
fig.subplots_adjust(top=0.8)
plt.suptitle('Missing value plot', size=30, y=1.0001);   

plt.show()

Drop missing value due to a big amount of null

In [ ]:
check_missing = [i for i in sales_train.isnull().sum().tolist() if i !=0] 
if len(check_missing) == 0 : print('No missing values in sales_train')

In [ ]:
msno.bar(prices)
fig.subplots_adjust(top=0.8)
plt.suptitle('Missing value plot', size=30, y=1.0001);   

plt.show()

# EDA

In [ ]:
plt.style.use("seaborn")
fig, axs = plt.subplots(1, 3, figsize = (10, 3))
fig.suptitle('Varied by categories')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
axs[0].hist(sales_train[sales_train['cat_id'] == 'HOBBIES'].groupby('cat_id').sum().T, color = 'y')
axs[1].hist(sales_train[sales_train['cat_id'] == 'FOODS'].groupby('cat_id').sum().T, color = 'r')
axs[2].hist(sales_train[sales_train['cat_id'] == 'HOUSEHOLD'].groupby('cat_id').sum().T, color = 'b')

axs[0].set_title('HOBBIES')
axs[1].set_title('FOODS')
axs[2].set_title('HOUSEHOLD')

plt.show()

fig, axs = plt.subplots(1, 3, figsize = (10, 3))
fig.suptitle('Varied by states')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
axs[0].hist(sales_train[sales_train['state_id'] == 'CA'].groupby('state_id').sum().T, color = 'y')
axs[1].hist(sales_train[sales_train['state_id'] == 'WI'].groupby('state_id').sum().T, color = 'r')
axs[2].hist(sales_train[sales_train['state_id'] == 'TX'].groupby('state_id').sum().T, color = 'b')

axs[0].set_title('CA')
axs[1].set_title('WI')
axs[2].set_title('TX')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

### First, We will select random goods at the same mall

In [ ]:
def select_state_cat(df, name):
    col = df.columns[6:]
    cal_col = ['d','date', 'wday', 'month', 'year']
    df  = df[df['id'] == name][col].T.reset_index()
    df['d'] = df['index']
    df.drop('index', axis = 1, inplace = True)
    
    s = df.columns[0]
    df = df.rename(columns = {s : 'sales'})
    
    df = pd.merge(df,calendar[cal_col], on = 'd')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index(df['date'], inplace = True)
    return df

#plot plot plot



select_state_cat(sales_train, 'HOBBIES_1_001_CA_1_evaluation')['sales'].plot(figsize=(10, 3),
          color='y',
          title='HOBBIES_1_001_CA_3 sales by actual sale dates')
plt.show()

select_state_cat(sales_train, 'FOODS_1_001_CA_1_evaluation')['sales'].plot(figsize=(10, 3),
          color='r',
          title='FOODS_1_001_CA_3 sales by actual sale dates')
plt.show()

select_state_cat(sales_train, 'HOUSEHOLD_1_001_CA_1_evaluation')['sales'].plot(figsize=(10, 3),
          color='b',
          title='HOUSEHOLD_1_001_CA_3 sales by actual sale dates')
plt.show()

    
    

- It's obvious that some products has zero-demand interval probably due to stockout

### Then, let's examine average sales by days of week

In [ ]:
select_state_cat(sales_train, 'HOBBIES_1_001_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='y',
          title='HOBBIES_1_001_CA_3 average sales by wday')
plt.show()

select_state_cat(sales_train, 'FOODS_1_001_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='r',
          title='FOODS_1_001_CA_3 average sales by wday')
plt.show()

select_state_cat(sales_train, 'HOUSEHOLD_1_001_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='b',
          title='HOUSEHOLD_1_001_CA_3 average sales by wday')
plt.show()

- For item 'HOUSEHOLD_1_001_CA_1' and 'FOODS_1_001_CA_1,We clearly see that there is a peak around weekend (wday = (1, 2)) and gradualy decay until wday = 4 (Tuesday) and then it starts to increase

### Let's examine another item

In [ ]:
select_state_cat(sales_train, 'HOBBIES_1_177_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='y',
          title='HOBBIES_1_177_CA_3 average sales by wday')
plt.show()

select_state_cat(sales_train, 'FOODS_1_177_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='r',
          title='FOODS_1_177_CA_3 average sales by wday')
plt.show()

select_state_cat(sales_train, 'HOUSEHOLD_1_177_CA_1_evaluation').groupby('wday').mean()['sales'].plot(figsize=(10, 3),
          color='b',
          title='HOUSEHOLD_1_177_CA_3 average sales by wday')
plt.show()

- Peak of this product's demand also be in the weekend and gradually decay through weekday


In [ ]:
select_state_cat(sales_train, 'HOBBIES_1_122_CA_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(10, 3),
          color='y',
          title='HOBBIES_1_177_CA_3 max sales by wday')
plt.show()


select_state_cat(sales_train, 'FOODS_1_122_CA_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(10, 3),
          color='r',
          title='FOODS_1_177_CA_3 max sales by wday')
plt.show()


select_state_cat(sales_train, 'HOUSEHOLD_1_122_CA_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(10, 3),
          color='b',
          title='HOUSEHOLD_1_177_CA_3 max sales by wday')
plt.show()


- Why does peak is in Wednesday and Friday (max weekly demand)?

In [ ]:
def merge_df(df):
    col = df.columns[6:]
    cal_col = ['d','date', 'wday', 'month', 'year']
    df  = df[col].T.reset_index()
    df['d'] = df['index']
    df.drop('index', axis = 1, inplace = True)
    
    s = df.columns[0]
    df = df.rename(columns = {s : 'sales'})
    
    df = pd.merge(df,calendar[cal_col], on = 'd')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index(df['date'], inplace = True)
    
    return df

data = merge_df(sales_train)

In [ ]:
sales_train.head()

In [ ]:
data.head()

In [ ]:
select_state_cat(sales_train, 'FOODS_1_122_CA_1_evaluation')[select_state_cat(sales_train, 'FOODS_1_122_CA_1_evaluation')['sales'] == 14]

In [ ]:
calendar.loc[(calendar['date'] == '2015-06-03')]

In [ ]:
select_state_cat(sales_train, 'FOODS_1_122_CA_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(12, 3),
          color='y',
          title='FOODS_1_122_CA_1 sales from CA max sales by wday')
plt.show()

select_state_cat(sales_train, 'FOODS_1_122_TX_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(12, 3),
          color='r',
          title='FOODS_1_122_TX_1 sales from TX max sales by wday')
plt.show()

select_state_cat(sales_train, 'FOODS_1_122_WI_1_evaluation').groupby('wday').max()['sales'].plot(figsize=(12, 3),
          color='b',
          title='FOODS_1_122_WI_1 sales from WI max sales by wday')
plt.show()

- จากการทดลองพบว่าวันที่ยอดขายอาหารสูงสุดสำหรับไอเทม 'FOODS_1_122_CA_1_evaluation' นั่นคือวันที่ 2015/06/03 ซึ่งเป็นวันที่รัฐบาลได้แจกจ่ายเงินสำหรับโปรแกรม SNAP หรือช่วยเหลือในด้านอาหารและโภชนการโดยสวัสดิการอันนี้รัฐจะแจกให้ช่วงวันที่ 1-14 โดย vary ตามเลขหลักสุดท้ายตามบัตร SSN ของคนอเมริกัน ซึ่งอาจเป็นการผลักดันให้คนออกไปซื้ออาหารกันมาก โดยในวันนี้ feature snap_CA เป็น 1 ตรงนี้น่าสนใจ
- และกราฟด้านบนยังสามารถทำให้เห็นได้ชัดว่าช่วงที่สินค้าอาหารขายดีที่สุดในแต่ละรัฐในวันที่ snap ของทุกรัฐเป็น 1 นั้น (วันที่ 2015/06/03 ตามหัวข้อด้านบน) จะเกิดในวันช่วงอังคารกับพุธนั่นเอง

- For item 'FOODS_1_122_CA_1_evaluation' in CA_1 branch, the peak is on 2015/6/3 because U.S. Government provide SNAP benefit as money in that day for low-benefit citizens to support their nutrition. The SNAP benefit is provided on different day with different state. 
- To summarize, the SNAP has an impact on food products.

source : 

https://www.fns.usda.gov/snap/supplemental-nutrition-assistance-program

https://www.masslegalservices.org/content/88-when-will-i-get-my-snap-benefits


In [ ]:
c = ['r', 'g', 'b', 'y']
for i in range(29,32):
    df = select_state_cat(sales_train, 'FOODS_1_0' + str(i) + '_CA_1_evaluation')
    df['day'] = df['date'].dt.day
    df.head()

    df.groupby('day').mean()['sales'].plot(figsize=(15, 5),
              
              title='FOODS sum sales by month in CA_1',
                                         label = 'ID :' + str(i),
                                         color = c[i-29])
plt.legend()
plt.show()

In [ ]:
# select_state_cat(sales_train, 'HOBBIES_1_122_CA_1_evaluation')['sales'].rolling(window=28).mean().loc[pd.Timestamp('2013-06-01'):].plot(figsize=(15, 8),
#           color='r',
#           title='Rolling average sales of HOBBIES_1_177_CA_3 compared with SNAP day',
#           linewidth=1)

# for i in list_of_snapCA_date:
#     plt.axvline(i,linewidth=0.1, color='g', alpha=0.5
list_of_snapCA_date = pd.to_datetime(calendar.loc[calendar['snap_CA'] != 0]['date'], format = '%Y-%m-%d').tolist()

import plotly.express as px

df = select_state_cat(sales_train, 'FOODS_1_029_CA_1_evaluation').rolling(window=28).mean()
df = df.reset_index()
fig = px.line(df, x="date", y="sales", title='Rolling average sales of HOBBIES_1_177_CA_3 compared with SNAP day')

for i in list_of_snapCA_date:
    fig.add_vline(x=i, line_width=0.5, line_dash="dash", line_color="green")
fig.show()

- กราฟนี้ช่วยสนับสนุนข้อคิดเห็นจากด้านบน เนื่องจากยอดขายสินค้าอาหารจะขายดีในช่วงวันที่ 1-14 ซึ่งเป็นวันที่รัฐจ่ายเงิน SNAP พอหลังจากวันที่ 14 ยอดขายสินค้าอาหารก็จะตกลงเรื่อยๆ


In [ ]:
select_state_cat(sales_train, 'HOBBIES_1_122_CA_1_evaluation').groupby('month').mean()['sales'].plot(figsize=(15, 3),
          color='y',
          title='HOBBIES_1_122_CA_1 average sales by month')
plt.show()


select_state_cat(sales_train, 'FOODS_1_122_CA_1_evaluation').groupby('month').mean()['sales'].plot(figsize=(15, 3),
          color='r',
          title='FOODS_1_122_CA_1 average sales by month')
plt.show()


select_state_cat(sales_train, 'HOUSEHOLD_1_122_CA_1_evaluation').groupby('month').mean()['sales'].plot(figsize=(15, 3),
          color='b',
          title='HOUSEHOLD_1_122_CA_1 average sales by month')
plt.show()

- กราฟด้านบนบ่งบอกว่าสินค้าจะขายดีช่วงเดือนธันวาคม ซึ่งเป็นช่วงคาบเกี่ยวระหว่างวันเทศกาลได้แก่วันคริสต์มาสและวันปีใหม่นั่นเอง
- Above plot show that the highest demand occurs in December (Christmas day and New year)

In [ ]:
df = sales_train[sales_train['id'].str.startswith('HOBBIES')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='y',
          title='HOBBIES sum sales')
plt.show()
del df

df = sales_train[sales_train['id'].str.startswith('FOODS')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='r',
          title='FOODS sum sales')
plt.show()
del df

df = sales_train[sales_train['id'].str.startswith('HOUSEHOLD')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='b',
          title='HOUSEHOLD sum sales')
plt.show()
del df

- จากกราฟด้านบน พบว่ายอดขายสินค้ารวมในแต่ละรัฐนั้น จะมียอดขายน้อยในช่วงก่อนเข้าคริสต์มาสต์ -> assume ว่าเก็บตังไว้รอคริสต์มาส?
- พฤติกรรมเป็นแบบนี้ในทุกรัฐจริงหรือ? ดูกราฟด้านล่าง

- Above plot is overall demand. There is seasonality in every year,

#### HOBBIES

In [ ]:
sales_train[(sales_train['cat_id'] == 'HOBBIES') & (sales_train['state_id'] == 'CA')]

In [ ]:
df = sales_train[(sales_train['cat_id'] == 'HOBBIES') & (sales_train['state_id'] == 'CA')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='y',
          title='HOBBIES sum sales in CA')
plt.show()
del df

df = sales_train[(sales_train['cat_id'] == 'HOBBIES') & (sales_train['state_id'] == 'WI')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='r',
          title='HOBBIES sum sales in WI')
plt.show()
del df

df = sales_train[(sales_train['cat_id'] == 'HOBBIES') & (sales_train['state_id'] == 'TX')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='b',
          title='HOBBIES sum sales in TX')
plt.show()
del df

#### FOODS

In [ ]:
df = sales_train[(sales_train['cat_id'] == 'FOODS') & (sales_train['state_id'] == 'CA')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='y',
          title='FOODS sum sales in CA')
plt.show()
del df

df = sales_train[(sales_train['cat_id'] == 'FOODS') & (sales_train['state_id'] == 'WI')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='r',
          title='FOODS sum sales in WI')
plt.show()
del df

df = sales_train[(sales_train['cat_id'] == 'FOODS') & (sales_train['state_id'] == 'TX')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='b',
          title='FOODS sum sales in TX')
plt.show()
del df

#### HOUSEHOLD

In [ ]:
df = sales_train.loc[(sales_train['id'].str.startswith('HOUSEHOLD')) | (sales_train['id'].str[-3:-1] == 'CA')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='y',
          title='HOUSEHOLD sum sales in CA')
plt.show()
del df

df = sales_train.loc[(sales_train['id'].str.startswith('HOUSEHOLD')) | (sales_train['id'].str[-3:-1] == 'WI')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='r',
          title='HOUSEHOLD sum sales in WI')
plt.show()
del df

df = sales_train.loc[(sales_train['id'].str.startswith('HOUSEHOLD')) | (sales_train['id'].str[-3:-1] == 'TX')][sales_train.columns[6:]].sum(axis = 0).reset_index()
df['d'] = df['index']
df = pd.merge(df, calendar[['d','date', 'wday', 'month', 'year']], on = 'd')
df = df[['date', 0]]
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.set_index('date', inplace = True)
df[0].plot(figsize=(15, 3),
          color='b',
          title='HOUSEHOLD sum sales in TX')
plt.show()
del df

In [ ]:
calendar[calendar['date'].str.endswith('12-25')]

- พฤติกรรมแบบนี้ทุกรัฐ

## Examine in mall-wise (same state)

อันนี้คืออยากรู้ว่าเทรนด์ของสินค้าแต่ละประเภทเป็นอย่างไร จะลองไอเทมเดียวกัน ที่วางขายในคนละห้างแต่อยู่รัฐเดียวกัน

In [ ]:
select_state_cat(sales_train, 'HOUSEHOLD_1_112_CA_1_evaluation').rolling(window=28).mean()['sales'].loc[pd.Timestamp('2015-12-01'):].plot(figsize=(15, 8),
          color='r',
          title='Rolling average sales of HOUSEHOLD_1_112 from different mall but same state',
          linewidth=1, label = 'CA_1')

select_state_cat(sales_train, 'HOUSEHOLD_1_112_CA_2_evaluation').rolling(window=28).mean()['sales'].loc[pd.Timestamp('2015-12-01'):].plot(figsize=(15, 8),
          color='g',
          
          linewidth=1, label = 'CA_2')

select_state_cat(sales_train, 'HOUSEHOLD_1_112_CA_3_evaluation').rolling(window=28).mean()['sales'].loc[pd.Timestamp('2015-12-01'):].plot(figsize=(15, 8),
          color='y',
          
          linewidth=1, label = 'CA_3')

plt.legend()
plt.show()

In [ ]:
sales_train.head()

In [ ]:
sales_train.loc[(sales_train['store_id'] == 'CA_1') & (sales_train['cat_id'] == 'HOBBIES')][sales_train.columns[6:]].sum(axis = 0).plot()

In [ ]:
sales_train.groupby(['state_id', 'cat_id']).sum()

def select_st_cat_rolling(df, store_name, cat_name, roll):
    df = df.loc[(df['store_id'] == store_name) & (df['cat_id'] == cat_name)]
    df = df[df.columns[6:]].sum(axis = 0).rolling(window=28).mean()
    
    return df


for i in ['HOBBIES', 'FOODS', 'HOUSEHOLD']:
    fig, axs = plt.subplots(1, 3, figsize=(16,5))
    
    fig.suptitle(i)
    
    for j in ['CA_', 'WI_', 'TX_']:
        
        for k in range(1,4):
            axs[k-1].plot(select_st_cat_rolling(sales_train, j+str(k), i, 28))
        
    plt.show()
        
# select_st_cat_rolling(sales_train, 'CA_1', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'CA_2', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'CA_3', 'HOBBIES', 28).plot()
# plt.show()

# select_st_cat_rolling(sales_train, 'WI_1', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'WI_2', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'WI_3', 'HOBBIES', 28).plot()
# plt.show()

# select_st_cat_rolling(sales_train, 'TX_1', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'TX_2', 'HOBBIES', 28).plot()
# select_st_cat_rolling(sales_train, 'TX_3', 'HOBBIES', 28).plot()
# plt.show()

# fig, (ax1, ax2) = plt.subplots(1, 2)
# fig.suptitle('Horizontally stacked subplots')
# ax1.plot(x, y)
# ax2.plot(x, -y)

- พบว่าสินค้าบางชิ้น ถ้าเป็นไอดีเดียวกันที่วางขายในห้างคนละห้างที่อยู่ในรัฐเดียวกัน มีแนวโน้มไปทางเดียวกัน อาจกล่าวได้ว่า สินค้าเดียวกันใดๆ ในรัฐเดียวกัน จะมีลักษณะเป็นเทรนด์เฉพาะของในแต่ละรัฐเป็น hub